In [2]:
# Данный ноутбук использовал окружение google-colab
%pip install catboost fasttext -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 101.2 kB/s eta 0:00:00


In [4]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 960.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 22.8 MB/s eta 0:00:00


# Домашнее задание "NLP. Часть 1"

In [24]:
import math
import re
import os
import random
import json
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Any
import random
import torch
import numpy as np
import datasets
import fasttext
import fasttext.util
from transformers import BertTokenizer, BertModel

In [6]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [7]:
def normalize_pretokenize_text(text: str) -> List[str]:
    text = text.lower()
    words = re.findall(r'\b\w+\b', text)
    return words

In [8]:
# This block is for tests only
test_corpus = [
    "the quick brown fox jumps over the lazy dog",
    "never jump over the lazy dog quickly",
    "brown foxes are quick and dogs are lazy"
]

def build_vocab(texts: List[str]) -> Tuple[List[str], Dict[str, int]]:
    all_words = []
    for text in texts:
        words = normalize_pretokenize_text(text)
        all_words.extend(words)
    vocab = sorted(set(all_words))
    vocab_index = {word: idx for idx, word in enumerate(vocab)}
    return vocab, vocab_index

vocab, vocab_index = build_vocab(test_corpus)

## Задание 1 (0.5 балла)
Реализовать One-Hot векторизацию текстов

In [25]:
def one_hot_vectorization(
    text: str,
    vocab: List[str] = None,
    vocab_index: Dict[str, int] = None
) -> List[List[int]]:
    words = normalize_pretokenize_text(text)
    result = []

    for word in words:
        vector = [0] * len(vocab)
        if word in vocab_index:
            idx = vocab_index[word]
            vector[idx] = 1
        result.append(vector)

    return result


def test_one_hot_vectorization(
    vocab: List[str],
    vocab_index: Dict[str, int]
) -> bool:
    try:
        text = "the quick brown fox"
        result = one_hot_vectorization(text, vocab, vocab_index)

        if not isinstance(result, list):
            return False

        expected_length = len(vocab)
        if len(result[0]) != expected_length:
            return False

        words_in_text = normalize_pretokenize_text(text)
        for i, word in enumerate(words_in_text):
            if word in vocab_index:
                idx = vocab_index[word]
                if result[i][idx] != 1:
                    return False

        print("One-Hot-Vectors test PASSED")

        return True
    except Exception as e:
        print(f"One-Hot-Vectors test FAILED: {e}")
        return False

In [26]:
assert test_one_hot_vectorization(vocab, vocab_index)

One-Hot-Vectors test PASSED


## Задание 2 (0.5 балла)
Реализовать Bag-of-Words

In [11]:
def bag_of_words_vectorization(text: str) -> Dict[str, int]:
    words = normalize_pretokenize_text(text)
    return dict(Counter(words))

def test_bag_of_words_vectorization() -> bool:
    try:
        text = "the the quick brown brown brown"
        result = bag_of_words_vectorization(text)

        if not isinstance(result, dict):
            return False

        if result.get('the', 0) != 2:
            return False
        if result.get('quick', 0) != 1:
            return False
        if result.get('brown', 0) != 3:
            return False
        if result.get('nonexistent', 0) != 0:
            return False

        print("Bad-of-Words test PASSED")
        return True
    except Exception as e:
        print(f"Bag-of-Words test FAILED: {e}")
        return False

In [12]:
assert test_bag_of_words_vectorization()

Bad-of-Words test PASSED


## Задание 3 (0.5 балла)
Реализовать TF-IDF

In [27]:
def tf_idf_vectorization(
    text: str,
    corpus: List[str] = None,
    vocab: List[str] = None,
    vocab_index: Dict[str, int] = None
) -> List[float]:
    words = normalize_pretokenize_text(text)
    word_counts = Counter(words)

    total_words = len(words)
    tf = {word: count / total_words for word, count in word_counts.items()}

    N = len(corpus)
    idf = {}

    for word in vocab:
        df = sum(1 for doc in corpus if word in normalize_pretokenize_text(doc))
        if df > 0:
            idf[word] = math.log(N / df)
        else:
            idf[word] = 0.0

    result = []
    for word in vocab:
        if word in tf:
            result.append(tf[word] * idf[word])
        else:
            result.append(0.0)

    return result


def test_tf_idf_vectorization(corpus, vocab, vocab_index) -> bool:
    try:
        text = "the quick brown"
        result = tf_idf_vectorization(text, corpus, vocab, vocab_index)

        if not isinstance(result, list):
            return False

        expected_length = len(vocab)
        if len(result) != expected_length:
            return False

        for val in result:
            if not isinstance(val, float):
                return False

        print("TF-IDF test PASSED")
        return True
    except Exception as e:
        print(f"TF-IDF test FAILED: {e}")
        return False

In [28]:
assert test_tf_idf_vectorization(test_corpus, vocab, vocab_index)

TF-IDF test PASSED


## Задание 4 (1 балл)
Реализовать Positive Pointwise Mutual Information (PPMI).  
https://en.wikipedia.org/wiki/Pointwise_mutual_information
$$PPMI(word, context) = max(0, PMI(word, context))$$
$$PMI(word, context) = log \frac{P(word, context)}{P(word) P(context)} = log \frac{N(word, context)|(word, context)|}{N(word) N(context)}$$
где $N(word, context)$ -- число вхождений слова $word$ в окно $context$ (размер окна -- гиперпараметр)

In [29]:
def ppmi_vectorization(
    text: str,
    corpus: List[str] = None,
    vocab: List[str] = None,
    vocab_index: Dict[str, int] = None,
    window_size: int = 2
) -> List[float]:
    word_counts = Counter()
    cooccurrence_counts = defaultdict(lambda: defaultdict(int))
    total_pairs = 0

    for doc in corpus:
        words = normalize_pretokenize_text(doc)
        word_counts.update(words)

        for i, word in enumerate(words):
            for j in range(max(0, i - window_size), min(len(words), i + window_size + 1)):
                if i != j:
                    context_word = words[j]
                    cooccurrence_counts[word][context_word] += 1
                    total_pairs += 1

    total_words = sum(word_counts.values())
    words = normalize_pretokenize_text(text)
    result = [0.0] * len(vocab)

    for word in words:
        if word not in word_counts:
            continue

        p_word = word_counts[word] / total_words

        for context_word in vocab:
            if context_word not in word_counts:
                continue

            p_context = word_counts[context_word] / total_words

            if total_pairs > 0:
                p_joint = cooccurrence_counts[word][context_word] / total_pairs
            else:
                p_joint = 0

            if p_joint > 0 and p_word > 0 and p_context > 0:
                pmi = math.log(p_joint / (p_word * p_context))
                ppmi = max(0, pmi)
            else:
                ppmi = 0.0

            idx = vocab_index[context_word]
            result[idx] += ppmi

    return result
def test_ppmi_vectorization(corpus, vocab, vocab_index) -> bool:
    try:
        text = "quick brown fox"
        result = ppmi_vectorization(text, corpus, vocab, vocab_index)

        if not isinstance(result, list):
            return False

        expected_length = len(vocab)
        if len(result) != expected_length:
            return False

        for val in result:
            if not isinstance(val, float):
                return False

        print("PPMI test PASSED")
        return True
    except Exception as e:
        print(f"PPMI test FAILED: {e}")
        return False

In [30]:
assert test_ppmi_vectorization(test_corpus, vocab, vocab_index)

PPMI test PASSED


## Задание 5 (1 балл)
Реализовать получение эмбеддингов из fasttext и bert (для bert лучше использовать CLS токен)

In [31]:
def get_fasttext_embeddings(
    text: str,
    model_path: str = None,
    model: any = None
) -> List[np.ndarray]:
    try:
        import fasttext
        import fasttext.util

        # Загружаем модель, если она не передана
        if model is None:
            if model_path is None:
                # Скачиваем модель по умолчанию
                fasttext.util.download_model('en', if_exists='ignore')
                model = fasttext.load_model('cc.en.300.bin')
            else:
                model = fasttext.load_model(model_path)

        words = normalize_pretokenize_text(text)
        embeddings = []

        for word in words:
            embedding = model.get_word_vector(word)
            embeddings.append(embedding)

        return embeddings

    except Exception as e:
        print(f"Error in fasttext embeddings: {e}")
        words = normalize_pretokenize_text(text)
        return [np.random.randn(300) for _ in words]


In [32]:
def get_bert_embeddings(
    text: str,
    model_name: str = 'bert-base-uncased',
    pool_method: str = 'cls'
) -> np.ndarray:

    try:
        from transformers import AutoTokenizer, AutoModel
        import torch

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)

        inputs = tokenizer(
            text,
            return_tensors='pt',
            max_length=512,
            truncation=True,
            padding=True
        )

        with torch.no_grad():
            outputs = model(**inputs)

        if pool_method == 'cls':
            embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        elif pool_method == 'mean':
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        elif pool_method == 'max':
            embedding = outputs.last_hidden_state.max(dim=1)[0].squeeze().numpy()
        else:
            embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

        return embedding

    except Exception as e:
        print(f"Error in BERT embeddings: {e}")
        return np.random.randn(768)

## Задание 6 (1.5 балла)
Реализовать обучение так, чтобы можно было поверх эмбеддингов, реализованных в предыдущих заданиях, обучить какую-то модель (вероятно неглубокую, например, CatBoost) на задаче классификации текстов ([IMDB](https://huggingface.co/datasets/stanfordnlp/imdb)).

In [34]:
def vectorize_dataset(
    dataset_name: str = "imdb",
    vectorizer_type: str = "bow",
    split: str = "train",
    sample_size: int = 2500
) -> Tuple[Any, List, List]:

    print(f"Loading {split} dataset with {vectorizer_type}...")

    dataset = datasets.load_dataset(dataset_name, split=split)

    if sample_size:
        indices = []
        labels_temp = [item['label'] for item in dataset]

        label_0_indices = [i for i, label in enumerate(labels_temp) if label == 0]
        label_1_indices = [i for i, label in enumerate(labels_temp) if label == 1]

        samples_per_class = min(sample_size // 2, len(label_0_indices), len(label_1_indices))

        random.seed(42)
        indices.extend(random.sample(label_0_indices, samples_per_class))
        indices.extend(random.sample(label_1_indices, samples_per_class))

        dataset = dataset.select(indices)

    texts = [item['text'] for item in dataset if 'text' in item and item['text'].strip()]
    labels = [item['label'] for item in dataset if 'label' in item]

    vocab, vocab_index = build_vocab(texts)

    vectorized_data = []

    for idx, text in enumerate(texts):
        if idx % 500 == 0:
            print(f"Processing {idx}/{len(texts)}...")

        try:
            if vectorizer_type == "one_hot":
                one_hot_vecs = one_hot_vectorization(text, vocab, vocab_index)
                if one_hot_vecs:
                    avg_vec = np.mean(one_hot_vecs, axis=0)
                    vectorized_data.append(avg_vec.tolist())
                else:
                    vectorized_data.append([0] * len(vocab))

            elif vectorizer_type == "bow":
                bow_dict = bag_of_words_vectorization(text)
                vector = [bow_dict.get(word, 0) for word in vocab]
                vectorized_data.append(vector)

            elif vectorizer_type == "tfidf":
                vectorized_data.append(tf_idf_vectorization(text, texts, vocab, vocab_index))

            elif vectorizer_type == "ppmi":
                vectorized_data.append(ppmi_vectorization(text, texts, vocab, vocab_index))

            elif vectorizer_type == "fasttext":
                embeddings = get_fasttext_embeddings(text)
                if embeddings:
                    avg_embedding = np.mean(embeddings, axis=0)
                    vectorized_data.append(avg_embedding.tolist())
                else:
                    vectorized_data.append([0] * 300)

            elif vectorizer_type == "bert":
                embedding = get_bert_embeddings(text)
                vectorized_data.append(embedding.tolist())

            else:
                raise ValueError(f"Unknown vectorizer type: {vectorizer_type}")

        except Exception as e:
            print(f"Error processing text {idx}: {e}")
            if vectorizer_type == "bert":
                vectorized_data.append([0] * 768)
            elif vectorizer_type == "fasttext":
                vectorized_data.append([0] * 300)
            else:
                vectorized_data.append([0] * len(vocab))

    print(f"Vectorization complete: {len(vectorized_data)} samples")
    return vocab, vectorized_data, labels

In [36]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold

def train(
    embeddings_method="bow",
    test_size=0.2,
    val_size=0.2,
    cv_folds=5
):

    print(f"\n{'='*60}")
    print(f"Training with {embeddings_method.upper()} embeddings")
    print(f"{'='*60}\n")

    try:
        vocab, X, y = vectorize_dataset("imdb", embeddings_method, "train")
        _, X_test, y_test = vectorize_dataset("imdb", embeddings_method, "test")

        X = np.array(X)
        y = np.array(y)
        X_test = np.array(X_test)
        y_test = np.array(y_test)

        print(f"Train shape: {X.shape}, Test shape: {X_test.shape}")
        print(f"Train labels distribution: {np.bincount(y)}")
        print(f"Test labels distribution: {np.bincount(y_test)}")

        if len(np.unique(y)) < 2:
            print("Error: Train set contains only one class!")
            return None

        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=val_size, random_state=42, stratify=y
        )

        print(f"Train: {X_train.shape}, Val: {X_val.shape}")
        print(f"Train val labels distribution: {np.bincount(y_train)}")
        print(f"Val labels distribution: {np.bincount(y_val)}")

        model = CatBoostClassifier(
            iterations=500,
            depth=6,
            learning_rate=0.1,
            loss_function='Logloss',
            random_seed=42,
            verbose=False,
            early_stopping_rounds=50
        )

        print("\nTraining model...")
        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            verbose=100
        )


        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)

        train_accuracy = accuracy_score(y_train, y_train_pred)
        val_accuracy = accuracy_score(y_val, y_val_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)

        train_f1 = f1_score(y_train, y_train_pred, average='binary')
        val_f1 = f1_score(y_val, y_val_pred, average='binary')
        test_f1 = f1_score(y_test, y_test_pred, average='binary')

        print(f"Results for {embeddings_method.upper()}")
        print(f"Train Accuracy: {train_accuracy:.4f} | F1: {train_f1:.4f}")
        print(f"Val   Accuracy: {val_accuracy:.4f} | F1: {val_f1:.4f}")
        print(f"Test  Accuracy: {test_accuracy:.4f} | F1: {test_f1:.4f}")

        print("Test Set Classification Report:")
        print(classification_report(y_test, y_test_pred, target_names=['Negative', 'Positive']))

    except Exception as e:
        print(f"Error during training: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
for embeddings_method in ["bow", "one_hot", "tfidf", "ppmi", "fasttext", "bert"]:
    train(embeddings_method=embeddings_method)


Training with BOW embeddings

Loading train dataset with bow...
Processing 0/2500...
Processing 500/2500...
Processing 1000/2500...
Processing 1500/2500...
Processing 2000/2500...
Vectorization complete: 2500 samples
Loading test dataset with bow...
Processing 0/2500...
Processing 500/2500...
Processing 1000/2500...
Processing 1500/2500...
Processing 2000/2500...
Vectorization complete: 2500 samples
Train shape: (2500, 28274), Test shape: (2500, 28010)
Train labels distribution: [1250 1250]
Test labels distribution: [1250 1250]
Train: (2000, 28274), Val: (500, 28274)
Train val labels distribution: [1000 1000]
Val labels distribution: [250 250]

Training model...
0:	learn: 0.6795156	test: 0.6776975	best: 0.6776975 (0)	total: 77.1ms	remaining: 38.5s
100:	learn: 0.3289552	test: 0.4322608	best: 0.4322608 (100)	total: 2.94s	remaining: 11.6s
200:	learn: 0.1950970	test: 0.3867040	best: 0.3867040 (200)	total: 5.79s	remaining: 8.62s
300:	learn: 0.1284985	test: 0.3669666	best: 0.3660941 (290)	t